# Where to move in? 
### (Toronto vs NYC neighborhoods)

![alt text](https://images.squarespace-cdn.com/content/v1/5d3c9dfcd2e1860001acb51d/1568582007983-1EUG6EDVM54MHIEMYZEU/ke17ZwdGBToddI8pDm48kPfW58HQawIq7Ev7Hj3E5hxZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpwZby2dJfdmC6uznuCz-gLnlBU1V3UIa9Xyt9ajd4Zc9pT0nIw4OBGb5aJH_SxB8e0/Toronto+Vs.+NYC "Logo Title Text 1")

## 1. Discussion and Background of the Business Problem:
### Problem Statement: Make long distance moving an easier decision

In these times of globalization is very usual that people moves from one city to another, even from one country to another for professional or academical reasons. For some, this decision is pleasant and a choice, but for some others is really hard and stressful. Leaving all the familiy, friends, traditions behind makes the task much more difficult. However, we could try to make this decision a bit easier by comparing multiple cities and find similarities of their neighborhoods in order to choose the one that will make you feel more comfortable. 

Between 1993 and 2013 USA was in the TOP 5 countries of immigration to Canada and Canada in the TOP 15 countries to USA. For this reason I find it representative to center the analysis in two cities: NYC and Toronto. So if you currently live in East Toronto, which NYC neighborhood would be the most suitable for you to move in? This is the question we will answer at the end of the analysis, but first let me walk you through all the steps of the project in this long journey from NYC to Toronto or viceversa.

## 2. Description of the data used for the analysis:
In this analysis I will use multiple datasets coming from different sources
- From the **UN**   
    *'International migration flows to and from selected countries: The 2015 revision.'* We will use USA and Canada data files only.   
	The dataset contains annual data on the flows of international migrants as recorded by the countries of destination. The data presents both inflows and outflows according to the place of birth, citizenship or place of previous / next residence both for foreigners and nationals. The current version presents data available from 45 countries.  
	source: https://www.un.org/en/development/desa/population/migration/data/empirical2/migrationflows.asp#
    
	
- From **NYU** (IBM downloaded version)  
	*'2014 New York City Neighborhood Names'*  
	To get a list of the NYC neighborhoods with coordinates.  
    source:https://geo.nyu.edu/catalog/nyu_2451_34572  
    source used: https://cocl.us/new_york_dataset
	
    
- From **Wikipedia**  
	*'List of postal codes of Canada: M.'*      
	It contains full list of the neighborhoods around Toronto by Postal Code  
    source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
	
    
- **Geocoder API**  
	To get the coordinates of Toronto neighborhoods and NYC
    
	
- **Foursquare venues API**  
	Will give us the list of venues for each neighborhood in order to make the comparison between them
    
#### STEPS TO FOLLOW
- First we will need to obtain the coordinates of NYC and Toronto neighborhoods.  
For that purpose we will use *2014 New York City Neighborhood Names*. In case of Toronto we will need an extra step cause coordinates are not available in the Wikipedia *List of postal codes of Canada: M.*. That's how we will use Geocoder to get the coordinates from the Postal Codes given in the list. After that, we will merge Toronto's and NYC coordinates list into the same dataset in order to manage alltogether.  
- With the list of centroids obtained in the previous step, we will run the Foursquare API in order to find all the venues located around those places 
- Afterwards by using the k-means algorithm we will determine the similarities on the neighborhoods

## 3. Data Preparation

First we need to install and import all the dependencies that we will use later. Make sure you uncomment the following lines in case you don't have GEOCODER and FOLIUM libraries installed.

In [ ]:
#GEOCODER
#!conda install -c conda-forge/label/cf202003 geocoder
#print('Geocoder installed successfully')

#FOLIUM
#!conda install -c conda-forge folium=0.5.0 --yes
#print('Folium installed successfully')

In [1]:
import pandas as pd
import numpy as np
import geocoder
import folium
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude


## Obtain data needed for the analysis

### New York City Neighborhoods

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
nyNeigh_data = newyork_data['features']    

Data downloaded!


In [5]:
#Load NY data into neighborhood DF
for data in nyNeigh_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
print(neighborhoods.shape)

(306, 4)


As there are many neighborhoods we will focus the analysis in Manhattan and Brooklyn boroughs

In [7]:
neighborhoods = neighborhoods[(neighborhoods['Borough']=='Manhattan')|(neighborhoods['Borough']=='Brooklyn')]
print(neighborhoods.shape)
neighborhoods.head()

(110, 4)


,Borough,Neighborhood,Latitude,Longitude
6,Manhattan,Marble Hill,40.876551,-73.910660
46,Brooklyn,Bay Ridge,40.625801,-74.030621
47,Brooklyn,Bensonhurst,40.611009,-73.995180
48,Brooklyn,Sunset Park,40.645103,-74.010316
49,Brooklyn,Greenpoint,40.730201,-73.954241


### Toronto Area Neighborhoods

To obtain the Toronto neighborhoods we will use the Wikipedia table for Postal Codes, then merge it with the latitude/longitude of each Postal Code from Geocoder

In [8]:
#Reading the Wikipedia table
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
CanadaPC = pd.read_html(link)[0]

#CLEANING THE TABLE
# Filters Not Assigend Borough
df_Canada = CanadaPC.loc[(CanadaPC['Borough'] != 'Not assigned')] 
# If there's no Neighbohood assigns Borough
df_Canada.Neighborhood.fillna(df_Canada.Borough, inplace=True)           
# Resets the index of the DF
df_Canada = df_Canada.reset_index(drop=True)                             
# Replaces '/' separator to ',' separtor for multiple neighborhoods
df_Canada['Neighborhood'].replace(to_replace ='/', value = ',', regex = True,inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [9]:
df_Canada.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [10]:
#For each postal code in previous step, we look for the coordinates
Can_Locations = pd.DataFrame(columns=['Postal_Code', 'Latitude', 'Longitude'])

for i in range(len(df_Canada)) :
    
    lat_lng_coords = None

    postal_code = df_Canada.loc[i, "Postal code"]
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    neighborhood_lat = lat_lng_coords[0]
    neighborhood_lon = lat_lng_coords[1]
    
    Can_Locations = Can_Locations.append({'Postal_Code': postal_code,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
Can_Locations.head()

,Postal_Code,Latitude,Longitude
0,M3A,43.752935,-79.335641
1,M4A,43.728102,-79.311890
2,M5A,43.650964,-79.353041
3,M6A,43.723265,-79.451211
4,M7A,43.661790,-79.389390


## Merge NYC and Toronto data


Now we append Toronto data into the df for neighborhoods (that already stores NYC data)

In [12]:
neighborhoods = neighborhoods.append(df_Canada.set_index('Postal code').join(Can_Locations.set_index('Postal_Code')))
neighborhoods.reset_index(drop=True)
print(neighborhoods.shape)
neighborhoods.head()

(213, 4)


,Borough,Neighborhood,Latitude,Longitude
6,Manhattan,Marble Hill,40.876551,-73.910660
46,Brooklyn,Bay Ridge,40.625801,-74.030621
47,Brooklyn,Bensonhurst,40.611009,-73.995180
48,Brooklyn,Sunset Park,40.645103,-74.010316
49,Brooklyn,Greenpoint,40.730201,-73.954241


## Map of the Locations that we are going to analyse further

In [210]:
#First we need to obtain Toronto's coords
g = geocoder.arcgis('Toronto, ON')
lat_lng_coords = g.latlng

lat_tor= lat_lng_coords[0]
long_tor = lat_lng_coords[1]

#Now we obtain NYC's coords
g = geocoder.arcgis('New York City, NY')
lat_lng_coords = g.latlng

lat_nyc= lat_lng_coords[0]
long_nyc = lat_lng_coords[1]

#Instances of both maps
map_city_tor = folium.Map(location=[lat_tor, long_tor], zoom_start=11,min_zoom=10, width='100%')
map_city_ny = folium.Map(location=[lat_nyc, long_nyc], zoom_start=12,min_zoom=11, width='100%')

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '({}), {}'.format(neighborhood.replace("'","&apos"), borough.replace("'","&apos"))
    label = folium.Popup(label, parse_html=True)
    
    if borough !='Manhattan' and borough !='Brooklyn':
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_city_tor)
    else:
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_city_ny)

In [212]:
#Displaying both maps side by side
from IPython.core.display import display, HTML

htmlcities = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 1px solid black"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 1px solid black"></iframe>'
           .format(map_city_tor.get_root().render().replace('"', '&quot;'),500,500,
                   map_city_ny.get_root().render().replace('"', '&quot;'),500,500))
display(htmlcities)

## Exploring Neighborhoods

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
neigh_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )
print(neigh_venues.shape)

(7965, 7)


In [16]:
print(neigh_venues.shape)
neigh_venues.head()

(7965, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [17]:
print('There are {} uniques categories.'.format(len(neigh_venues['Venue Category'].unique())))

There are 416 uniques categories.


#### As there are many venues & categories we will transform the data into one hot encoding and group venues by neighborhood given the mean of the frequency of occurrence of each category

In [18]:
# one hot encoding
neigh_onehot = pd.get_dummies(neigh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neigh_onehot['Neighborhood'] = neigh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neigh_onehot.columns[-1]] + list(neigh_onehot.columns[:-1])
neigh_onehot = neigh_onehot[fixed_columns]

neigh_grouped = neigh_onehot.groupby('Neighborhood').sum().reset_index()
neigh_grouped.head(10)

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Animal Shelter,Antique Shop,...,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Alderwood , Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bath Beach,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,"Bathurst Manor , Wilson Heights , Downsview North",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Battery Park City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
5,Bay Ridge,1,0,0,0,0,0,3,0,0,...,1,0,1,0,0,0,0,0,0,0
6,Bayview Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Bedford Park , Lawrence Manor East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Bedford Stuyvesant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
9,Bensonhurst,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### We will determine the top 10 categories for each Neighborhood

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neigh_grouped['Neighborhood']

for ind in np.arange(neigh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neigh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Badminton Court,Supermarket,Sushi Restaurant,Skating Rink,Factory,Electronics Store,Elementary School,Empanada Restaurant,English Restaurant
1,"Alderwood , Long Branch",Athletics & Sports,Sandwich Place,Convenience Store,Pharmacy,Pub,Coffee Shop,Gym,Gas Station,Pizza Place,Dance Studio
2,Bath Beach,Pharmacy,Chinese Restaurant,Sushi Restaurant,Fast Food Restaurant,Bubble Tea Shop,Asian Restaurant,Italian Restaurant,Gas Station,Donut Shop,Bagel Shop
3,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Supermarket,Sandwich Place,Restaurant,Fried Chicken Joint,Shopping Mall,Deli / Bodega,Pharmacy,Sushi Restaurant
4,Battery Park City,Park,Hotel,Gym,Boat or Ferry,Memorial Site,Coffee Shop,Plaza,Wine Shop,Food Court,Beer Garden


In [21]:
print('There are {} uniques categories.'.format(len(np.unique(neighborhoods_venues_sorted.iloc[:,1:(num_top_venues+1)].values))))

There are 248 uniques categories.


In [22]:
X = neigh_grouped.loc[:,np.unique(neighborhoods_venues_sorted.iloc[:,1:(num_top_venues+1)].values)].values
X = np.nan_to_num(X)

In [23]:
clusterNum = 12
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 20)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[ 8  8  8  8 10  5  8  8  1  8  1  8  8  2  8  8  8  1  2  8  2  6  1  8
  6  8  6  5  8  6  8 11  2  8  6  8 10  8  8 10  0  2  9  8  8  8  1  8
  8  2  8  8  8  2  8  1  8  8  2  8  7  2  8  8  1 10  9  8 10  8  8  1
  2  1  6  8  8  8  8  2  2  8  0  5  8  0  8  8  8  8  8 10  8  8  8  2
  8  8  8  1  8  1  8  8  0 10  5  1  1  8  8  8  8  1  1  8  8  6  4  8
  8  8  8  8  8  1  2  8  8  5  8  7  8  8  8  8  8  8  8  1  6  8  8  2
  2  8  6  1  8  8  6  8  8  8  8  0  8  8  8  8  5  7  8  6  1  8  8  6
  8  8  8  8 10  8  8  8  8  8  9 10  2 10  1  5  5  8  1  2  8  8  5  8
  8  8  1  1  8  1  8  8  8  8  8  3]


In [201]:
neigh_grouped["Cluster"] = labels
neighborhoods_venues_sorted["Cluster"] = labels

cities_clustered = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cities_clustered = cities_clustered.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
cities_clustered.dropna(axis=0, inplace=True)
print(cities_clustered.shape)
cities_clustered.head()

(209, 15)


,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
6,Manhattan,Marble Hill,40.876551,-73.910660,Sandwich Place,American Restaurant,Gym,Coffee Shop,Yoga Studio,Department Store,Miscellaneous Shop,Steakhouse,Supplement Shop,Seafood Restaurant,8.0
46,Brooklyn,Bay Ridge,40.625801,-74.030621,Italian Restaurant,Spa,Pizza Place,Greek Restaurant,Bar,American Restaurant,Thai Restaurant,Middle Eastern Restaurant,Bagel Shop,Sushi Restaurant,5.0
47,Brooklyn,Bensonhurst,40.611009,-73.995180,Italian Restaurant,Ice Cream Shop,Grocery Store,Donut Shop,Chinese Restaurant,Sushi Restaurant,Hotpot Restaurant,Bank,Bar,Liquor Store,8.0
48,Brooklyn,Sunset Park,40.645103,-74.010316,Mexican Restaurant,Pizza Place,Latin American Restaurant,Bakery,Bank,Fried Chicken Joint,Mobile Phone Shop,Gym,Bagel Shop,Supplement Shop,8.0
49,Brooklyn,Greenpoint,40.730201,-73.954241,Pizza Place,Bar,Coffee Shop,Cocktail Bar,Yoga Studio,Deli / Bodega,Café,Sushi Restaurant,Polish Restaurant,Bakery,0.0


In [25]:
# set color scheme for the clusters
x = np.arange(clusterNum)
ys = [i + x + (i*x)**2 for i in range(clusterNum)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [182]:
# create NYC map
lat_nyc = 40.741349
long_nyc= -73.992644
map_nyc = folium.Map(location=[lat_nyc, long_nyc], zoom_start=12,min_zoom=11, width='100%')


# add markers to the map
for lat, lon, poi, cluster, borough in zip(cities_clustered['Latitude'], cities_clustered['Longitude'], cities_clustered['Neighborhood'], cities_clustered['Cluster'],cities_clustered['Borough']):
    label = folium.Popup(str(poi), parse_html=True)
    
    if borough =='Manhattan' or  borough =='Brooklyn':
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_nyc)
       
    
#map_nyc

In [202]:
# create maps
lat_tor = 43.701023
long_tor = -79.380847
map_toronto = folium.Map(location=[lat_tor, long_tor], zoom_start=11,min_zoom=10, width='100%')

# add markers to the map
for lat, lon, poi, cluster, borough in zip(cities_clustered['Latitude'], cities_clustered['Longitude'], cities_clustered['Neighborhood'], cities_clustered['Cluster'],cities_clustered['Borough']):
    label = folium.Popup(str(poi.replace("'","&apos")), parse_html=True)
    
    if borough !='Manhattan' and borough !='Brooklyn':
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_toronto)


In [198]:
cities_clustered.tail()

,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster
M8X,Etobicoke,The Kingsway Montgomery Road Old Mill North,43.653340,-79.509766,Park,Pool,Lounge,Smoke Shop,Women's Store,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,8.0
M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Pub,Grocery Store,Dance Studio,Café,Hotel,Smoke Shop,6.0
M7Y,East Toronto,Business reply mail Processing CentrE,43.648700,-79.385450,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Asian Restaurant,Sushi Restaurant,Bookstore,Italian Restaurant,Tea Room,6.0
M8Y,Etobicoke,Old Mill South King's Mill Park Sunnylea Hu...,43.632798,-79.493017,Park,Bank,Construction & Landscaping,Home Service,Exhibit,Duty-free Shop,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,8.0
M8Z,Etobicoke,Mimico NW The Queensway West South of Bloor ...,43.625490,-79.526000,Burrito Place,Thai Restaurant,Middle Eastern Restaurant,Gym,Gym / Fitness Center,Coffee Shop,Mattress Store,Sushi Restaurant,Bank,Miscellaneous Shop,8.0


In [213]:
from IPython.core.display import display, HTML


htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 1px solid black"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 1px solid black"></iframe>'
           .format(map_toronto.get_root().render().replace('"', '&quot;'),500,500,
                   map_nyc.get_root().render().replace('"', '&quot;'),500,500))
display(htmlmap)